# k3sx

Install k3s on given ssh remotes via k3sup and setup additional components for a proper production cluster.

## Prerequesites

To proceed, you need one or more exposed remote Linux hosts, which you can access via public IPv4 and can login to as a privileged user via a private key.

## Setup

### 1. Supply all inputs

#### Set environment variables for SSH

In [ ]:
# user name and ssh key for login on your remotes
%env SSH_USER=root
%env SSH_KEY=~/.ssh/id_rsa

#### Create txt files and fill them with the IPv4 addresses of your remotes

One IP per line

In [ ]:
%%bash

touch ./server-ips.txt
touch ./agent-ips.txt

if command -v code &> /dev/null
then 
    code ./server-ips.txt
    code ./agent-ips.txt
fi

#### Copy values.yaml template to root and fill in all parameters

In [ ]:
%%bash

if [ ! -f ./values.yaml ]; 
then
    cp ./chart/values.yaml ./values.yaml
fi

if command -v code &> /dev/null
then
    code ./values.yaml
fi

### 2. Install CLI tools

#### Quick install via Homebrew

Install [brew](https://brew.sh/) if you don't have it yet: 

```bash 
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
echo 'export PATH="/home/linuxbrew/.linuxbrew/bin:/home/linuxbrew/.linuxbrew/sbin:$PATH"' >> ~/.profile
``` 

This requires sudo and must be done outside of this notebook.

In [ ]:
%%bash
brew install kubernetes-cli int128/kubelogin/kubelogin helm k3sup fluxcd/tap/flux vcluster cilium-cli

### 3. Setup k3s on all given ssh remotes

In [ ]:
%%bash

for SERVER in $(cat ./server-ips.txt)
do
    ssh-keygen -f "$HOME/.ssh/known_hosts" -R "$SERVER"
    scp -o StrictHostKeyChecking=no -i $SSH_KEY ./config/k3s.yaml $SSH_USER@$SERVER:~/
    scp -i $SSH_KEY ./config/90-override.conf $SSH_USER@$SERVER:/etc/sysctl.d/ # Ubuntu 21+ quirk with Cilium
    # See https://github.com/cilium/cilium/issues/18131#issuecomment-988160016
    ssh -i $SSH_KEY $SSH_USER@$SERVER "echo 'PubkeyAcceptedKeyTypes=+ssh-rsa' >> /etc/ssh/sshd_config & reboot" # Ubuntu 21+ quirk with k3sup
    sleep 30
    k3sup install --ip $SERVER --user $SSH_USER --ssh-key $SSH_KEY --local-path ./kubeconfig \
        --k3s-extra-args '--config ~/k3s.yaml --node-taint node.cilium.io/agent-not-ready=true:NoExecute'
    export JOIN_SERVER=$SERVER
done

In [ ]:
%%bash
for AGENT in $(cat ./agent-ips.txt)
do
    ssh-keygen -f "$HOME/.ssh/known_hosts" -R $AGENT
    scp -o StrictHostKeyChecking=no -i $SSH_KEY ./config/90-override.conf $SSH_USER@$AGENT:/etc/sysctl.d/ # Ubuntu 21+ quirk with Cilium
    ssh -i $SSH_KEY $SSH_USER@$AGENT "echo 'PubkeyAcceptedKeyTypes=+ssh-rsa' >> /etc/ssh/sshd_config & reboot" # Ubuntu 21+ quirk with k3sup
    sleep 30
    k3sup join --ip $AGENT --server-ip 49.12.239.191 --user $SSH_USER --ssh-key $SSH_KEY \
        --k3s-extra-args '--node-taint node.cilium.io/agent-not-ready=true:NoExecute'
done

In [ ]:
%env KUBECONFIG=./kubeconfig

### 4. Install necessary infrastructure to run pods

Some components must be deployed even before FluxCD to be able to run pods (with storage). These manual deployments will be reconciled with a GitOps deployment later.

In [ ]:
%%bash

helm dependency build charts/bootstrap/cilium
helm install cilium charts/bootstrap/cilium --namespace kube-system

helm dependency build charts/bootstrap/longhorn
helm install longhorn charts/bootstrap/longhorn --namespace longhorn-system --create-namespace

sleep 60 # wait for pods to be ready

#### Confirm that all nodes have status *Ready*

In [ ]:
%%bash
kubectl get node -o wide

#### Check Cilium networking status

In [ ]:
%%bash
cilium status

#### Change the default storage class to Longhorn

In [ ]:
%%bash
kubectl patch storageclass local-path -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"false"}}}'
kubectl patch storageclass longhorn -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}'

### 5. [Bootstrap FluxCD](https://fluxcd.io/docs/installation/#bootstrap) 

In [ ]:
%%bash
flux install

#### Alternative

If you wish to have the entire cluster be controlled my a single GitOps repo, instead do [`flux bootstrap github`](https://fluxcd.io/docs/installation/#github-and-github-enterprise) for automatically setting up a GitOps repo on GitHub or [`flux bootstrap git`](https://fluxcd.io/docs/installation/#generic-git-server) for using an existing GitOps repo with an arbitrary provider.

### 6. Install the main infrastructure chart

**NOTE:** If you chose to setup a GitOps repo for the entire cluster in the previous step, create and commit respective YAML definitions of [GitRepository](https://fluxcd.io/docs/guides/helmreleases/#git-repository) and [HelmRelease](https://fluxcd.io/docs/guides/helmreleases/#define-a-helm-release) resources to your repo instead of using the CLI commands shown below.

#### Create a GitRepository source from this repo

In [ ]:
%%bash

flux create source git k3sx-root \
  --url=https://github.com/lorenzo-w/k3sx \
  --branch=main

#### Create a HelmRelease with values from ./values.yaml

In [ ]:
%%bash

flux create hr k3sx-main \
  --source=GitRepository/k3sx-root \
  --chart=./charts/main \
  --values=./values.yaml \
sleep 300 # wait for all sub-releases to be reconciled, yes this may take up to 5 min

#### Confirm that all HelmRelease reconciliations are done and successful

Otherwise wait a couple more seconds and refresh.

In [ ]:
%%bash
flux get hr --all-namespaces

And also confirm that no errors have occured

In [ ]:
%%bash
flux logs --level error -n flux-system

### 7. Activate OIDC login for Kubernetes and set up kubelogin for kubectl

In [ ]:
%%bash
# Add OIDC to the kube api server

ISSUER_URL=$(kubectl get secret sso-config -n sso-system --template={{.data.oidc_issuer_url}} | base64 --decode)
CLIENT_ID=$(kubectl get secret sso-config -n sso-system --template={{.data.internal_oidc_client_id}} | base64 --decode)
CLIENT_SECRET=$(kubectl get secret sso-config -n sso-system --template={{.data.internal_oidc_client_secret}} | base64 --decode)

for SERVER in $(cat ./server-ips.txt)
do
    ssh -i $SSH_KEY $SSH_USER@$SERVER "rm -f /var/lib/rancher/k3s/server/token" # This file causes trouble if present on k3s restart
    k3sup install --ip $SERVER --user $SSH_USER --ssh-key $SSH_KEY --local-path ./kubeconfig --k3s-extra-args "--config ~/k3s.yaml --kube-apiserver-arg=oidc-issuer-url=$ISSUER_URL --kube-apiserver-arg=oidc-client-id=$CLIENT_ID --kube-apiserver-arg=oidc-groups-claim=groups"
done

In [ ]:
%%bash
# Add OIDC to local kubeconfig file

ISSUER_URL=$(kubectl get secret sso-config -n sso-system --template={{.data.oidc_issuer_url}} | base64 --decode)
CLIENT_ID=$(kubectl get secret sso-config -n sso-system --template={{.data.internal_oidc_client_id}} | base64 --decode)
CLIENT_SECRET=$(kubectl get secret sso-config -n sso-system --template={{.data.internal_oidc_client_secret}} | base64 --decode)

kubectl config set-credentials oidc \
  --exec-api-version=client.authentication.k8s.io/v1beta1 \
  --exec-command=kubectl \
  --exec-arg=oidc-login \
  --exec-arg=get-token \
  --exec-arg=--oidc-issuer-url=$ISSUER_URL \
  --exec-arg=--oidc-client-id=$CLIENT_ID \
  --exec-arg=--oidc-client-secret=$CLIENT_SECRET